In [1]:
from UCIDataset import UCIDataset
from EvolutionaryWrapperFeatureSelection import EvolutionaryWrapperFeatureSelection
from SurrogateAssistedWrapperFeatureSelection import SurrogateAssistedWrapperFeatureSelection
from sklearn.datasets import make_classification
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Synthetic Data
Feature selection is done for synthetic data of which the informative features are known. In the below example a binary classification dataset of 1000 instances are created of which only the first two features (indexes [0,1]) are informative and the rest are random noise. Three wrapper feature selection methods are used to identify the infromative features using a Decision Tree classifier:
<ul>
<li>SAGA</li>
<li>CHC</li>
<li>GA</li>
</ul>

In [2]:
def createDataset(n_samples, n_features, n_classes, n_informative, classifier):
    X, y = make_classification(n_samples=n_samples, n_features=n_features, n_classes=n_classes,
                               n_informative=n_informative, n_redundant=0, n_repeated=0,
                               random_state=1, shuffle=False)

    df = pd.DataFrame(X)
    df['labels'] = pd.Series(y)
    label = -1
    header = None

    dataset = UCIDataset(df, 'df', label, divide_dataset=False, header=header)
    dataset.divideDataset(classifier,
                          normalize=True, 
                          shuffle=True, 
                          all_features=True, 
                          all_instances=True, 
                          evaluate=True, 
                          partial_sample=False)

    print('Number of classes:', n_classes)
    print('Number of instances:', n_samples)
    print('Number of features:', n_features)
    print('Informative features:', list(range(0, n_informative)))
    accuracy = np.round(100* dataset.getTestAccuracy(), 2)
    print('Test accuracy before feature selection: %', accuracy)



    
    return dataset

In [23]:
dataset = createDataset(1000, 100, 2, 2, DecisionTreeClassifier(random_state=0))
alpha = 0.88
verbose = 0
evaluation = 'validation'

Number of classes: 2
Number of instances: 1000
Number of features: 100
Informative features: [0, 1]
Test accuracy before feature selection: % 76.0


# SAGA

In [24]:
log, population = SurrogateAssistedWrapperFeatureSelection.SAGA(dataset, 
                                                                populationSize=40, 
                                                                a=16, 
                                                                reductionRate=0.5, 
                                                                step=10, 
                                                                alpha=alpha,
                                                                verbose=verbose,
                                                                evaluation=evaluation,
                                                                noChange=100,
                                                                timeout=np.inf)

feature_subset = log.iloc[-1]['best_solution']  
feature_subset = np.array(feature_subset)

elapsed_time = np.round(log.iloc[-1]['time'], 2)
accuracy = np.round(100*EvolutionaryWrapperFeatureSelection.evaluate(feature_subset, 'feature_selection', 'test', dataset, 1)[0], 2)

print('Test accuracy: %', accuracy)
print('Computation time: ', elapsed_time, 'sec')
print('Selected features indexes: ', np.where(feature_subset == 1)[0])

Test accuracy: % 84.5
Computation time:  15.3 sec
Selected features indexes:  [0 1]


# CHC

In [25]:
log, population = EvolutionaryWrapperFeatureSelection.CHC(dataset, 
                                                          populationSize=40, 
                                                          d=False, 
                                                          divergence=0.35, 
                                                          alpha=alpha,
                                                          evaluation=evaluation,
                                                          maxNochange=100,
                                                          verbose=verbose,
                                                          timeout=np.inf)

feature_subset = log.iloc[-1]['best_solution']  
feature_subset = np.array(feature_subset)

elapsed_time = np.round(log.iloc[-1]['time'], 2)
accuracy = np.round(100*EvolutionaryWrapperFeatureSelection.evaluate(feature_subset, 'feature_selection', 'test', dataset, 1)[0], 2)

print('Test accuracy: %', accuracy)
print('Computation time: ', elapsed_time, 'sec')
print('Selected features indexes: ', np.where(feature_subset == 1)[0])

Test accuracy: % 84.5
Computation time:  23.48 sec
Selected features indexes:  [0 1]


# GA

In [ ]:
log, population = EvolutionaryWrapperFeatureSelection.GA(dataset, 
                                                         populationSize=40, 
                                                         crossOverP=0.9, 
                                                         mutationP=0.1,
                                                         alpha=alpha,
                                                         evaluation=evaluation,
                                                         maxNochange=100,
                                                         verbose=verbose,
                                                         timeout=np.inf)
feature_subset = log.iloc[-1]['best_solution']  
feature_subset = np.array(feature_subset)

elapsed_time = np.round(log.iloc[-1]['time'], 2)
accuracy = np.round(100*EvolutionaryWrapperFeatureSelection.evaluate(feature_subset, 'feature_selection', 'test', dataset, 1)[0], 2)

print('Test accuracy: %', accuracy)
print('Computation time: ', elapsed_time, 'sec')
print('Selected features indexes: ', np.where(feature_subset == 1)[0])

In [13]:
comparision_df = pd.DataFrame(columns=('experiment', 'algorithm', 'accuracy', 'total_time'))
experiment_counter = 0

for i in range(50):
    
    log, population = SurrogateAssistedWrapperFeatureSelection.SAGA(dataset, 
                                                                populationSize=40, 
                                                                a=16, 
                                                                reductionRate=0.5, 
                                                                step=10, 
                                                                alpha=alpha,
                                                                verbose=verbose,
                                                                evaluation='validation',
                                                                noChange=100,
                                                                timeout=np.inf)

    feature_subset = log.iloc[-1]['best_solution']  
    feature_subset = np.array(feature_subset)

    elapsed_time = np.round(log.iloc[-1]['time'], 2)
    accuracy = np.round(100*EvolutionaryWrapperFeatureSelection.evaluate(feature_subset, 'feature_selection', 'test', dataset, alpha)[0], 2)


    
    row = [experiment_counter, 'SAGA', EvolutionaryWrapperFeatureSelection.hammingDistance(feature_subset, best_ind), elapsed_time]  
    comparision_df.loc[len(comparision_df)] = row
    print(row)
    
    log, population = EvolutionaryWrapperFeatureSelection.CHC(dataset, 
                                                          populationSize=40, 
                                                          d=False, 
                                                          divergence=0.35, 
                                                          alpha=alpha,
                                                          evaluation='validation',
                                                          maxNochange=np.inf,
                                                          verbose=verbose,
                                                          timeout=elapsed_time)

    feature_subset = log.iloc[-1]['best_solution']  
    feature_subset = np.array(feature_subset)

    elapsed_time = np.round(log.iloc[-1]['time'], 2)
    accuracy = np.round(100*EvolutionaryWrapperFeatureSelection.evaluate(feature_subset, 'feature_selection', 'test', dataset, alpha)[0], 2)

    row = [experiment_counter, 'CHC', EvolutionaryWrapperFeatureSelection.hammingDistance(feature_subset, best_ind), elapsed_time]  
    comparision_df.loc[len(comparision_df)] = row
    print(row)
    
    log, population = EvolutionaryWrapperFeatureSelection.GA(dataset, 
                                                         populationSize=40, 
                                                         crossOverP=0.9, 
                                                         mutationP=0.1,
                                                         alpha=alpha,
                                                         evaluation='validation',
                                                         maxNochange=np.inf,
                                                         verbose=verbose,
                                                         timeout=elapsed_time)
    feature_subset = log.iloc[-1]['best_solution']  
    feature_subset = np.array(feature_subset)

    elapsed_time = np.round(log.iloc[-1]['time'], 2)
    accuracy = np.round(100*EvolutionaryWrapperFeatureSelection.evaluate(feature_subset, 'feature_selection', 'test', dataset, alpha)[0], 2)

    row = [experiment_counter, 'GA', EvolutionaryWrapperFeatureSelection.hammingDistance(feature_subset, best_ind), elapsed_time]  
    comparision_df.loc[len(comparision_df)] = row
    print(row)
    
    experiment_counter +=1
    sns_plot= sns.catplot(x='algorithm', y='accuracy', kind="box", data=comparision_df)
    plt.show()
    


[0, 'SAGA', 8.0, 40.56]
[0, 'CHC', 6.0, 40.72]


KeyboardInterrupt: 